In [4]:
# =============================
# STEP 1: Upload & Unzip Dataset
# =============================
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()
zip_path = '/content/archive.zip'  # Adjust if different
extract_path = '/content/dataset'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check dataset folder structure
!ls dataset

Saving archive.zip to archive.zip
garbage_classification


In [5]:
# Check the directory structure within /content/dataset
#!ls -R /content/dataset


# Check if there's an extra folder layer
#extracted_items = os.listdir('dataset')
#print("Extracted items:", extracted_items)

# Confirm inner folder
#dataset_root = os.path.join('dataset', 'garbage_classification')
#print("Dataset Root:", dataset_root)

In [6]:
import tensorflow as tf
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Step 3: Setup ImageDataGenerator for Training & Validation
img_height, img_width = 150, 150
batch_size = 16

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    extract_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    extract_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 12412 images belonging to 1 classes.
Found 3103 images belonging to 1 classes.


In [7]:
# Step 4: Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification: paper or not
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 82944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,308,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,327,937 (20.32 MB)

 Trainable params: 5,327,937 (20.32 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Step 5: Train the Model
epochs = 1
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

776/776 ━━━━━━━━━━━━━━━━━━━━ 475s 612ms/step - accuracy: 1.0000 - loss: 1.4669e-06 - val_accuracy: 1.0000 - val_loss: 9.8918e-07


In [15]:
# Step 6: Save the Model
saved_model_path = '/content/saved_model_directory'
os.makedirs(saved_model_path, exist_ok=True)

# Save the model with the recommended .keras extension
model.save(saved_model_path + '/my_model.keras')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# Assuming IMG_SIZE and num_classes are already defined
# Example (adjust according to your dataset):
# IMG_SIZE = (224, 224)
# num_classes = 6 # Number of garbage categories in your dataset

# Build the InceptionResNetV2 Model
base_model = InceptionResNetV2(include_top=False, input_shape=target_size + (3,), weights='imagenet')
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = tf.keras.applications.inception_resnet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

inception_model = Model(inputs, outputs)

# Compile the Model
inception_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Train the Model
epochs = 4  # You can adjust the number of epochs
history_incep = inception_model.fit(train_ds, validation_data=val_ds, epochs=epochs)